# Importamos los archivos necesarios


In [1]:
import spacy
# Import functions
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
# !wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/sentence_processing/anomaly_threat_sentences.py
# !wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/sentence_processing/asset_threat_impact_prob_sentences.py
# !wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/sentence_processing/asset_threats_sentences.py
# !wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/threat_risk_sentences.py

# Import files
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/anomalies.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/threats.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/risks.txt
# Import dictionaries
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/dictionaries/anomalies_dict.json
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/dictionaries/threats_dict.json
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/dictionaries/assets_dict.json
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/dictionaries/risks_dict.json
# import explacy
!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py
from functions import *
import explacy

--2021-05-27 15:35:10--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7643 (7.5K) [text/plain]
Saving to: ‘functions.py’

functions.py        100%[===================>]   7.46K  --.-KB/s    in 0s      

2021-05-27 15:35:10 (69.5 MB/s) - ‘functions.py’ saved [7643/7643]

--2021-05-27 15:35:10--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/anomalies.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 226 

# Definición de funciones
En lugar de importar las funciones las definimos directamente así podemos hacer cambios fácilmente

In [2]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])
def process_anomaly_threat_sentence(sentence: str, result: {}):
  """Function that process a sentence for finding the terms prob, anomalyType, 
  threatType and impact""" 
  
  sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."

  doc = nlp(str(sentence))

  anomaly = extract_subject(sentence)
  anomaly = clean(anomaly)
  anomaly_matches = search_matches(anomalies_dict, anomaly)
  anomaly_count = counter(anomaly_matches)
  anomaly = higher_frecuency_term(anomaly_count)
  result["anomalyType"] = anomaly

  threat = extract_dobject(sentence)
  threat = clean(threat)
  threat_matches = search_matches(threats_dict, threat)  
  threat_count = counter(threat_matches)
  # print(threat_count)  
  threat = higher_frecuency_term(threat_count)  
  result["threatType"] = threat

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)
  
  return result

def process_asset_threat_impact_prob_sentence(sentence: str, result: {}):
  """Function that process a sentences finding the terms:
  asset, threatType, prob and impact"""
  sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."
  doc = nlp(str(sentence)) 
  
  asset = extract_passive_subject(sentence)      
  asset = clean(asset)
  asset_matches = search_matches(assets_dict, asset)
  asset_count = counter(asset_matches)
  asset = higher_frecuency_term(asset_count)
  result["asset"] = asset


  threat = extract_threat_passive(sentence)
  threat = clean(threat)
  threat_matches = search_matches(threats_dict, threat)  
  threat_count = counter(threat_matches)  
  threat = higher_frecuency_term(threat_count)  
  result["threatType"] = threat

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)
  return result

def process_asset_threats_sentence(sentence: str, result: {}):
  """Function that process a sentences finding the terms:
  asset, threatType1, threatType2, prob and impact"""

  sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."
  doc = nlp(str(sentence)) 

  asset = extract_dobject(sentence)      
  asset = clean(asset)
  asset_matches = search_matches(assets_dict, asset)
  asset_count = counter(asset_matches)
  asset = higher_frecuency_term(asset_count)
  result["asset"] = asset


  raw_threats = extract_subject(sentence)
  threats= []
  aux = []
  for element in raw_threats:
      if element == "," or element == "and":
          threats.append(aux)
          aux = []
      elif element == raw_threats[-1]:
          threats.append(aux)
          
      else:
          aux.append(element)


  threatsList = []
  for threat in threats:
      threat = clean(threat)
      threat_matches = search_matches(threats_dict, threat)
      threat_count = counter(threat_matches)
      threat = higher_frecuency_term(threat_count)
      threatsList.append(threat)
      result["threatType"] = threatsList

  # threat = clean(threat)
  # threat_matches = search_matches(threats_dict, threat)  
  # threat_count = counter(threat_matches)  
  # threat = higher_frecuency_term(threat_count)  
  # result["threatType"] = threat

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)
  return result

def process_threats_risks_sentence(sentence: str, result: {}):
    """Function that process a sentence for finding the terms prob, anomalyType, 
    threatType and impact""" 

    # sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."

    doc = nlp(str(sentence))

    raw_risks = extract_dobject(sentence)
    # print(raw_risks)
    risks= []
    aux = []
    for element in raw_risks:
        if element == "," or element == "and":
            risks.append(aux)
            aux = []
        elif element == raw_risks[-1]:
            risks.append(aux)
        else:
            aux.append(element)

    
    risksList = []
    for risk in risks:
        risk = clean(risk)
        risk_matches = search_matches(risks_dict, risk)
        risk_count = counter(risk_matches)
        risk = higher_frecuency_term(risk_count)
        risksList.append(risk)
        result["riskType"] = risksList

    threat = extract_subject(sentence)
    threat = clean(threat)
    threat_matches = search_matches(threats_dict, threat)  
    threat_count = counter(threat_matches)  
    threat = higher_frecuency_term(threat_count)  
    result["threatType"] = threat

    # result["prob"] = extract_prob(sentence)
    # result["impact"] = extract_impact(sentence)

    return result
    # print(doc)
    # print(result)

# Test de combinación de términos 


# `process_anomaly_threat_sentence`

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
anomalies_dict = read_json('/content/anomalies_dict.json')
anomalies = load_doc('/content/anomalies.txt')
# seleccionamos verbo y completamos la frase
verb = "generates"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

anomalies_list = []
for line in anomalies:
  clean_line = line.strip()
  anomalies_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "anomalyType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for anomaly in anomalies:
  anomaly = anomaly.strip()
  # convertimos la anómalia en palabras que se pueden usar en lenguaje natural
  anomalywords = anomaly.split("_") 
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "anomalyType": anomaly}

    # sintagma anomalía
    separate_anomaly = ''
    for word in anomalywords:
      if word.lower() != 'anomaly':
        separate_anomaly = separate_anomaly + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_anomaly.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_anomaly_threat_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')


 detected generates accidents with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'Accidents', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected generates bad reputation threat with probability 4 and impact 5

 detected generates compliance threat with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'ComplianceThreat', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected generates data protection risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DataProtectionRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected generates other legal obligation risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'OtherLegalObligationRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatTyp

In [ ]:

sentence = "uba sensor generates human resources not available with probability 4 and impact 5"
result = {}
result = process_anomaly_threat_sentence(sentence,result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree        Token       Dep type Lemma       Part of Sp
─────────────── ─────────── ──────── ─────────── ──────────
            ┌─► uba         compound uba         PROPN     
         ┌─►└── sensor      nsubj    sensor      NOUN      
┌────────┼───── generates   ROOT     generate    VERB      
│        │  ┌─► human       amod     human       ADJ       
│        └─►└── resources   dobj     resource    NOUN      
│           ┌─► not         neg      not         PART      
└─►┌────────┴── available   oprd     available   ADJ       
   └─►┌──────── with        prep     with        ADP       
      └─►┌─┬┬── probability pobj     probability NOUN      
         │ │└─► 4           nummod   4           NUM       
         │ └──► and         cc       and         CCONJ     
         └─►┌── impact      conj     impact      NOUN      
            └─► 5           nummod   5           NUM       
{'anomalyType': 'UBA_Sensor_Anomaly', 'threatType': 'HumanThreat', 'prob': [4], 'impact': [5]}


#`process_asset_threat_impact_prob_sentence`


In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
assets_dict = read_json('/content/assets_dict.json')
assets = load_doc('/content/assets.txt')
# seleccionamos verbo y completamos la frase
verb = "is exposed to"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

assets_list = []
for line in assets:
  clean_line = line.strip()
  assets_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "asset": ""
  }

# iteración de todas las anomalías y todas las amenazas
for asset in assets:
  asset = asset.strip()
  assetwords = camel_case_split(asset)
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "asset": asset}

    # sintagma anomalía
    separate_asset = ''
    for word in assetwords:
      if word.lower() != 'asset':
        separate_asset = separate_asset + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_asset.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_asset_threat_impact_prob_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
{'prob': [4], 'impact': [5], 'threatType': 'DeliberatedMaliciousSWDistribution', 'asset': 'RealEstate'}

 real estate is exposed to deliberated threat with probability 4 and impact 5

 real estate is exposed to deliberated unauthorized access with probability 4 and impact 5

 real estate is exposed to device theft with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DeviceTheft', 'asset': 'RealEstate'}
{'prob': [4], 'impact': [5], 'threatType': '', 'asset': 'RealEstate'}

 real estate is exposed to denial of service with probability 4 and impact 5

 real estate is exposed to device lost with probability 4 and impact 5

 real estate is exposed to economic loss with probability 4 and impact 5

 real estate is exposed to fire with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'Fire', 'asset': 'RealEstate'}
{'prob': [4], 'impact': [5], 'threatType': '', 'asset': 'RealEstate'}

 real 

#`process_asset_threats`


In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
assets_dict = read_json('/content/assets_dict.json')
assets = load_doc('/content/assets.txt')
# seleccionamos verbo y completamos la frase
verb = "threatens"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

assets_list = []
for line in assets:
  clean_line = line.strip()
  assets_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "asset": ""
  }

# iteración de todas las anomalías y todas las amenazas
for asset in assets:
  asset = asset.strip()
  assetwords = camel_case_split(asset)
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": [threat],
      "asset": asset}

    # sintagma anomalía
    separate_asset = ''
    for word in assetwords:
      if word.lower() != 'asset':
        separate_asset = separate_asset + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_threat.lower() +  ' ' + verb  + separate_asset.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_asset_threats_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 configuration error threatens raster image file extension with probability 4 and impact 5

 corporate brand image damages threatens raster image file extension with probability 4 and impact 5

 delayed delivery threatens raster image file extension with probability 4 and impact 5

 deliberated config files tampering threatens raster image file extension with probability 4 and impact 5

 deliberated hw tampering threatens raster image file extension with probability 4 and impact 5

 deliberated information destruction threatens raster image file extension with probability 4 and impact 5

 deliberated information leak threatens raster image file extension with probability 4 and impact 5

 deliberated information tampering threatens raster image file extension with probability 4 and impact 5

 deliberated malicious sw distribution threatens raster image file extension with probability 4 and impact 5

 deliberated registers tamp

# `process_threats_risks_sentence`

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
risks_dict = read_json('/content/risks_dict.json')
risks = load_doc('/content/risks.txt')
# seleccionamos verbo y completamos la frase
verb = "generates"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

risks_list = []
for line in risks:
  clean_line = line.strip()
  risks_list.append(clean_line)

result = {
    "threatType": "",
    "riskType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for risk in risks:
  risk = risk.strip()
  riskwords = camel_case_split(risk)
  for threat in threats_dict:
    
    expected = {
      "threatType": threat,
      "riskType": [risk]}

    # sintagma anomalía
    separate_risk = ''
    for word in riskwords:
      if word.lower() != 'risk':
        separate_risk= separate_risk + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence =  separate_threat.lower() + ' ' + verb  + separate_risk.lower() 
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_threats_risks_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 unexpected usage generates fire
{'threatType': 'UnexpectedUsage', 'riskType': ['FireRisk']}
{'threatType': '', 'riskType': ['']}

 natural disasters generates fire
{'threatType': 'NaturalDisasters', 'riskType': ['FireRisk']}
{'threatType': 'NaturalDisasters', 'riskType': ['']}

 non intentional admin error generates fire
{'threatType': 'NonIntentionalAdminError', 'riskType': ['FireRisk']}
{'threatType': 'NonIntentionalAdminError', 'riskType': ['']}

 non intentional information destruction generates fire
{'threatType': 'NonIntentionalInformationDestruction', 'riskType': ['FireRisk']}
{'threatType': 'NonIntentionalInformationDestruction', 'riskType': ['']}

 non intentional information leak generates fire
{'threatType': 'NonIntentionalInformationLeak', 'riskType': ['FireRisk']}
{'threatType': 'NonIntentionalInformationLeak', 'riskType': ['']}

 non intentional information tampering generates fire
{'threatType': 'NonIntentiona

# Test cambiando verbos

# `process_anomaly_threat_sentence` verbo **to produce**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
anomalies_dict = read_json('/content/anomalies_dict.json')
anomalies = load_doc('/content/anomalies.txt')
# seleccionamos verbo y completamos la frase
verb = "produces"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

anomalies_list = []
for line in anomalies:
  clean_line = line.strip()
  anomalies_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "anomalyType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for anomaly in anomalies:
  anomaly = anomaly.strip()
  # convertimos la anómalia en palabras que se pueden usar en lenguaje natural
  anomalywords = anomaly.split("_") 
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "anomalyType": anomaly}

    # sintagma anomalía
    separate_anomaly = ''
    for word in anomalywords:
      if word.lower() != 'anomaly':
        separate_anomaly = separate_anomaly + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_anomaly.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_anomaly_threat_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')


 detected produces accidents with probability 4 and impact 5

 detected produces bad reputation threat with probability 4 and impact 5

 detected produces compliance threat with probability 4 and impact 5

 detected produces data protection risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DataProtectionRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': 'StrategicPlanOnRisk', 'anomalyType': 'Detected_Anomaly'}

 detected produces other legal obligation risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'OtherLegalObligationRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': 'StrategicPlanOnRisk', 'anomalyType': 'Detected_Anomaly'}

 detected produces configuration error with probability 4 and impact 5

 detected produces corporate brand image damages with probability 4 and impact 5

 detected produces delayed delivery with probability 4 and impact 5
{'prob

# `process_anomaly_threat_sentence` verbo **to cause**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
anomalies_dict = read_json('/content/anomalies_dict.json')
anomalies = load_doc('/content/anomalies.txt')
# seleccionamos verbo y completamos la frase
verb = "causes"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

anomalies_list = []
for line in anomalies:
  clean_line = line.strip()
  anomalies_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "anomalyType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for anomaly in anomalies:
  anomaly = anomaly.strip()
  # convertimos la anómalia en palabras que se pueden usar en lenguaje natural
  anomalywords = anomaly.split("_") 
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "anomalyType": anomaly}

    # sintagma anomalía
    separate_anomaly = ''
    for word in anomalywords:
      if word.lower() != 'anomaly':
        separate_anomaly = separate_anomaly + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_anomaly.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_anomaly_threat_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')


 detected causes accidents with probability 4 and impact 5

 detected causes bad reputation threat with probability 4 and impact 5

 detected causes compliance threat with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'ComplianceThreat', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected causes data protection risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DataProtectionRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected causes other legal obligation risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'OtherLegalObligationRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': 'StrategicPlanOnRisk', 'anomalyType': 'Detected_Anomaly'}

 detected causes configuration error with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType

# `process_anomaly_threat_sentence` verbo **to induce**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
anomalies_dict = read_json('/content/anomalies_dict.json')
anomalies = load_doc('/content/anomalies.txt')
# seleccionamos verbo y completamos la frase
verb = "induces"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

anomalies_list = []
for line in anomalies:
  clean_line = line.strip()
  anomalies_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "anomalyType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for anomaly in anomalies:
  anomaly = anomaly.strip()
  # convertimos la anómalia en palabras que se pueden usar en lenguaje natural
  anomalywords = anomaly.split("_") 
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "anomalyType": anomaly}

    # sintagma anomalía
    separate_anomaly = ''
    for word in anomalywords:
      if word.lower() != 'anomaly':
        separate_anomaly = separate_anomaly + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_anomaly.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_anomaly_threat_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')


 detected induces accidents with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'Accidents', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected induces bad reputation threat with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'BadReputationThreat', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected induces compliance threat with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'ComplianceThreat', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected induces data protection risks with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DataProtectionRisks', 'anomalyType': 'Detected_Anomaly'}
{'prob': [4], 'impact': [5], 'threatType': '', 'anomalyType': ''}

 detected induces other legal obligation risks with probability 4 an

#`process_asset_threat_impact_prob_sentence` verbo **is discovered to** 


In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
assets_dict = read_json('/content/assets_dict.json')
assets = load_doc('/content/assets.txt')
# seleccionamos verbo y completamos la frase
verb = "is discovered to"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

assets_list = []
for line in assets:
  clean_line = line.strip()
  assets_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "asset": ""
  }

# iteración de todas las anomalías y todas las amenazas
for asset in assets:
  asset = asset.strip()
  assetwords = camel_case_split(asset)
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "asset": asset}

    # sintagma anomalía
    separate_asset = ''
    for word in assetwords:
      if word.lower() != 'asset':
        separate_asset = separate_asset + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_asset.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_asset_threat_impact_prob_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 real estate is discovered to non intentional information destruction with probability 4 and impact 5

 real estate is discovered to non intentional information leak with probability 4 and impact 5

 real estate is discovered to non intentional information tampering with probability 4 and impact 5

 real estate is discovered to non intentional malicious sw distribution with probability 4 and impact 5

 real estate is discovered to non intentional user error with probability 4 and impact 5

 real estate is discovered to partnerships issues with probability 4 and impact 5

 real estate is discovered to physical threat with probability 4 and impact 5

 real estate is discovered to terrorism with probability 4 and impact 5

 real estate is discovered to power outage with probability 4 and impact 5

 real estate is discovered to stakeholders issues with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'Stakeho

In [ ]:
sentence = "computers is exhibited to denial of service with probability 4 and impact 5"
result = {}
result = process_asset_threat_impact_prob_sentence(sentence, result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree                Token       Dep type  Lemma       Part of Sp
─────────────────────── ─────────── ───────── ─────────── ──────────
                   ┌──► computers   nsubjpass computer    NOUN      
                   │┌─► is          auxpass   be          AUX       
┌┬┬────────────────┴┴── exhibited   ROOT      exhibit     VERB      
││└─►┌───────────────── to          prep      to          PART      
││   └─►┌────────────── denial      pobj      denial      VERB      
││      └─►┌─────────── of          prep      of          ADP       
││         └─►┌──────── service     pobj      service     NOUN      
││            └─►┌───── with        prep      with        ADP       
││               └─►┌── probability pobj      probability NOUN      
││                  └─► 4           nummod    4           NUM       
│└────────────────────► and         cc        and         CCONJ     
└──────────────────►┌── impact      conj      impact      NOUN      
                    └─► 5         

#`process_asset_threat_impact_prob_sentence` verbo **is exhibited to** 


In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
assets_dict = read_json('/content/assets_dict.json')
assets = load_doc('/content/assets.txt')
# seleccionamos verbo y completamos la frase
verb = "is exhibited to"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

assets_list = []
for line in assets:
  clean_line = line.strip()
  assets_list.append(clean_line)

result = {
    "prob": 0,
    "impact": 0,
    "threatType": "",
    "asset": ""
  }

# iteración de todas las anomalías y todas las amenazas
for asset in assets:
  asset = asset.strip()
  assetwords = camel_case_split(asset)
  for threat in threats_dict:
    
    expected = {
      "prob": [4],
      "impact": [5],
      "threatType": threat,
      "asset": asset}

    # sintagma anomalía
    separate_asset = ''
    for word in assetwords:
      if word.lower() != 'asset':
        separate_asset = separate_asset + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence = separate_asset.lower() + ' ' + verb  + separate_threat.lower() + ' ' + pandimp
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_asset_threat_impact_prob_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 real estate is exhibited to deliberated threat with probability 4 and impact 5

 real estate is exhibited to deliberated unauthorized access with probability 4 and impact 5

 real estate is exhibited to device theft with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'DeviceTheft', 'asset': 'RealEstate'}
{'prob': [4], 'impact': [5], 'threatType': '', 'asset': 'RealEstate'}

 real estate is exhibited to denial of service with probability 4 and impact 5

 real estate is exhibited to device lost with probability 4 and impact 5

 real estate is exhibited to economic loss with probability 4 and impact 5

 real estate is exhibited to fire with probability 4 and impact 5
{'prob': [4], 'impact': [5], 'threatType': 'Fire', 'asset': 'RealEstate'}
{'prob': [4], 'impact': [5], 'threatType': '', 'asset': 'RealEstate'}

 real estate is exhibited to hw maintenance error with probability 4 and impact 5

 real estate i

In [ ]:
sentence = "computers is exhibited to denial of service with probability 4 and impact 5"
result = {}
result = process_asset_threat_impact_prob_sentence(sentence, result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree                Token       Dep type  Lemma       Part of Sp
─────────────────────── ─────────── ───────── ─────────── ──────────
                   ┌──► computers   nsubjpass computer    NOUN      
                   │┌─► is          auxpass   be          AUX       
┌┬┬────────────────┴┴── exhibited   ROOT      exhibit     VERB      
││└─►┌───────────────── to          prep      to          PART      
││   └─►┌────────────── denial      pobj      denial      VERB      
││      └─►┌─────────── of          prep      of          ADP       
││         └─►┌──────── service     pobj      service     NOUN      
││            └─►┌───── with        prep      with        ADP       
││               └─►┌── probability pobj      probability NOUN      
││                  └─► 4           nummod    4           NUM       
│└────────────────────► and         cc        and         CCONJ     
└──────────────────►┌── impact      conj      impact      NOUN      
                    └─► 5         

# `process_threats_risks_sentence` verbo **to produce**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
risks_dict = read_json('/content/risks_dict.json')
risks = load_doc('/content/risks.txt')
# seleccionamos verbo y completamos la frase
verb = "produces"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

risks_list = []
for line in risks:
  clean_line = line.strip()
  risks_list.append(clean_line)

result = {
    "threatType": "",
    "riskType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for risk in risks:
  risk = risk.strip()
  riskwords = camel_case_split(risk)
  for threat in threats_dict:
    
    expected = {
      "threatType": threat,
      "riskType": [risk]}

    # sintagma anomalía
    separate_risk = ''
    for word in riskwords:
      if word.lower() != 'risk':
        separate_risk= separate_risk + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence =  separate_threat.lower() + ' ' + verb  + separate_risk.lower() 
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_threats_risks_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 delayed delivery produces partnership
{'threatType': 'DelayedDelivery', 'riskType': ['PartnershipRisk']}
{'threatType': 'DelayedDelivery', 'riskType': ['']}

 deliberated config files tampering produces partnership
{'threatType': 'DeliberatedConfigFilesTampering', 'riskType': ['PartnershipRisk']}
{'threatType': '', 'riskType': ['']}

 deliberated hw tampering produces partnership
{'threatType': 'DeliberatedHWTampering', 'riskType': ['PartnershipRisk']}
{'threatType': '', 'riskType': ['']}

 deliberated information destruction produces partnership
{'threatType': 'DeliberatedInformationDestruction', 'riskType': ['PartnershipRisk']}
{'threatType': 'DeliberatedInformationDestruction', 'riskType': ['']}

 deliberated information leak produces partnership
{'threatType': 'DeliberatedInformationLeak', 'riskType': ['PartnershipRisk']}
{'threatType': 'DeliberatedInformationLeak', 'riskType': ['']}

 deliberated information tampering p

In [ ]:

sentence = "accidents generates Users Complaints Risks"
result = {}
result = process_threats_risks_sentence(sentence, result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree Token      Dep type Lemma     Part of Sp
──────── ────────── ──────── ───────── ──────────
    ┌─►  accidents  nsubj    accident  NOUN      
┌───┴──  generates  ROOT     generate  VERB      
│  ┌──►  Users      compound user      NOUN      
│  │┌─►  Complaints compound complaint NOUN      
└─►└┴──  Risks      dobj     risk      NOUN      
{'riskType': ['AccidentRisk'], 'threatType': 'Accidents'}


# `process_threats_risks_sentence` verbo **to cause**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
risks_dict = read_json('/content/risks_dict.json')
risks = load_doc('/content/risks.txt')
# seleccionamos verbo y completamos la frase
verb = "causes"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

risks_list = []
for line in risks:
  clean_line = line.strip()
  risks_list.append(clean_line)

result = {
    "threatType": "",
    "riskType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for risk in risks:
  risk = risk.strip()
  riskwords = camel_case_split(risk)
  for threat in threats_dict:
    
    expected = {
      "threatType": threat,
      "riskType": [risk]}

    # sintagma anomalía
    separate_risk = ''
    for word in riskwords:
      if word.lower() != 'risk':
        separate_risk= separate_risk + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence =  separate_threat.lower() + ' ' + verb  + separate_risk.lower() 
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_threats_risks_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
 deliberated threat causes physical failure
{'threatType': 'DeliberatedThreat', 'riskType': ['PhysicalFailureRisk']}
{'threatType': '', 'riskType': ['LogicalFailureRisk']}

 deliberated unauthorized access causes physical failure
{'threatType': 'DeliberatedUnauthorizedAccess', 'riskType': ['PhysicalFailureRisk']}
{'threatType': 'DeliberatedUnauthorizedAccess', 'riskType': ['LogicalFailureRisk']}

 device theft causes physical failure
{'threatType': 'DeviceTheft', 'riskType': ['PhysicalFailureRisk']}
{'threatType': 'DeviceTheft', 'riskType': ['LogicalFailureRisk']}

 denial of service causes physical failure
{'threatType': 'DenialOfService', 'riskType': ['PhysicalFailureRisk']}
{'threatType': 'DenialOfService', 'riskType': ['LogicalFailureRisk']}

 device lost causes physical failure
{'threatType': 'DeviceLost', 'riskType': ['PhysicalFailureRisk']}
{'threatType': 'DeviceTheft', 'riskType': ['LogicalFailureRisk']}

 economic lo

In [ ]:

sentence = "accidents generates Users Complaints Risks"
result = {}
result = process_threats_risks_sentence(sentence, result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree Token      Dep type Lemma     Part of Sp
──────── ────────── ──────── ───────── ──────────
    ┌─►  accidents  nsubj    accident  NOUN      
┌───┴──  generates  ROOT     generate  VERB      
│  ┌──►  Users      compound user      NOUN      
│  │┌─►  Complaints compound complaint NOUN      
└─►└┴──  Risks      dobj     risk      NOUN      
{'riskType': ['AccidentRisk'], 'threatType': 'Accidents'}


# `process_threats_risks_sentence` verbo **to induce**

In [ ]:
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

#importamos diccionarios de términos 
threats_dict = read_json('/content/threats_dict.json')
risks_dict = read_json('/content/risks_dict.json')
risks = load_doc('/content/risks.txt')
# seleccionamos verbo y completamos la frase
verb = "induces"
pandimp = "with probability 4 and impact 5"
# parámetros que calcularemos
errors = 0
total = 0
from random import *

risks_list = []
for line in risks:
  clean_line = line.strip()
  risks_list.append(clean_line)

result = {
    "threatType": "",
    "riskType": ""
  }

# iteración de todas las anomalías y todas las amenazas
for risk in risks:
  risk = risk.strip()
  riskwords = camel_case_split(risk)
  for threat in threats_dict:
    
    expected = {
      "threatType": threat,
      "riskType": [risk]}

    # sintagma anomalía
    separate_risk = ''
    for word in riskwords:
      if word.lower() != 'risk':
        separate_risk= separate_risk + " " + word
    
    # sintagma amenaza
    threat = camel_case_split(threat)
    separate_threat = ''
    for word in threat:
      separate_threat = separate_threat + " " + word

    # creamos la oración
    sentence =  separate_threat.lower() + ' ' + verb  + separate_risk.lower() 
    print()
    print(sentence)
    
    # se procesa la oración
    result = process_threats_risks_sentence(sentence, result)
    # se comparan los resultados y se pintan los resultados en caso de que error
    if expected != result:
      print(expected)
      print(result)
      errors += 1
    total +=1

print(f'errores: {errors} total: {total} tasa de error: {((errors/total)*100)} %')

Se han truncado las últimas 5000 líneas del flujo de salida.
{'threatType': 'ICTThreat', 'riskType': ['']}

 identity thief induces partnership
{'threatType': 'IdentityThief', 'riskType': ['PartnershipRisk']}
{'threatType': 'IdentityThief', 'riskType': ['']}

 monitoring error induces partnership
{'threatType': 'MonitoringError', 'riskType': ['PartnershipRisk']}
{'threatType': 'MonitoringError', 'riskType': ['']}

 network outage induces partnership
{'threatType': 'NetworkOutage', 'riskType': ['PartnershipRisk']}
{'threatType': '', 'riskType': ['']}

 physical fault induces partnership
{'threatType': 'PhysicalFault', 'riskType': ['PartnershipRisk']}
{'threatType': 'PhysicalFault', 'riskType': ['']}

 privilege escalation induces partnership
{'threatType': 'PrivilegeEscalation', 'riskType': ['PartnershipRisk']}
{'threatType': 'PrivilegeEscalation', 'riskType': ['']}

 sw maintenance error induces partnership
{'threatType': 'SWMaintenanceError', 'riskType': ['PartnershipRisk']}
{'threatT

In [ ]:

sentence = "accidents generates Users Complaints Risks"
result = {}
result = process_threats_risks_sentence(sentence, result)
nlp = spacy.load('en')
explacy.print_parse_info(nlp, sentence)
print(result)

Dep tree Token      Dep type Lemma     Part of Sp
──────── ────────── ──────── ───────── ──────────
    ┌─►  accidents  nsubj    accident  NOUN      
┌───┴──  generates  ROOT     generate  VERB      
│  ┌──►  Users      compound user      NOUN      
│  │┌─►  Complaints compound complaint NOUN      
└─►└┴──  Risks      dobj     risk      NOUN      
{'riskType': ['AccidentRisk'], 'threatType': 'Accidents'}
